# Assignment 5: Model Selection for KNN

In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In this assignment, you will continue practicing the fifth step of the machine learning life cycle and perform model selection to find the best performing KNN model for a classification problem.

You will complete the following tasks:

1. Build your DataFrame and define your ML problem
3. Create labeled examples from the data set
4. Split the data into training and test data sets
5. Perform a grid search to identify the optimal value of $K$ for a KNN classifier
6. Fit the optimal KNN classifier to the training data and make predictions on the test data
7. Evaluate the accuracy of the model
8. Plot a precision-recall curve for the model


<b>Note</b>: Some of the evaluation metrics we will be using are suited for binary classification models that produce probabilities. For this reason, we will be using the `predict_proba()` method to produce class label probability predictions. Recall that KNN is *not* a probabilistic method. Because of this, `predict_proba()` does not output true probabilities. What it does is the following: For n_neighbors=$k$, it identifies the closest $k$ points to a given input point. It then counts up the likelihood, among these $k$ points, of belonging to one of the classes and uses that as the class "probabilities." We will be using KNN for the sake of demonstrating how to use these evaluation metrics.

**<font color='red'>Note: Some of the code cells in this notebook may take a while to run.</font>**

## Part 1. Build Your DataFrame and Define Your ML Problem

#### Load a Data Set and Save it as a Pandas DataFrame

We will work with the "cell2celltrain" data set. This version of the data set has been preprocessed and is ready for modeling.

In [ ]:
# Do not remove or edit the line below:
filename = os.path.join(os.getcwd(), "data_KNN", "cell2celltrain.csv")

**Task**: Load the data and save it to DataFrame `df`.

In [1]:
import pandas as pd
import os

filename = os.path.join(os.getcwd(), "data_KNN", "cell2celltrain.csv")

df = pd.read_csv(filename)

print(df.head())

   CustomerID  Churn  ChildrenInHH  HandsetRefurbished  HandsetWebCapable  \
0     3000002   True         False               False               True   
1     3000010   True          True               False              False   
2     3000014  False          True               False              False   
3     3000022  False         False               False               True   
4     3000026   True         False               False              False   

   TruckOwner  RVOwner  HomeownershipKnown  BuysViaMailOrder  \
0       False    False                True              True   
1       False    False                True              True   
2       False    False               False             False   
3       False    False                True              True   
4       False    False                True              True   

   RespondsToMailOffers  ...  Occupation_Crafts  Occupation_Homemaker  \
0                  True  ...                0.0                   0.0   
1     

#### Define the Label

This is a binary classification problem in which we will predict customer churn. The label is the `Churn` column.

#### Identify Features

Our features will be all of the remaining columns in the dataset.

## Part 2. Create Labeled Examples from the Data Set 

<b>Task</b>: In the code cell below, create labeled examples from DataFrame `df`.

In [2]:
X = df.drop('Churn', axis=1)  
y = df['Churn']  
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (51047, 84)
Shape of y: (51047,)


## Part 3. Create Training and Test Data Sets
<b>Task</b>: In the code cell below, create training and test sets out of the labeled examples. Create a test set that is 10 percent of the size of the data set.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (45942, 84)
Shape of X_test: (5105, 84)
Shape of y_train: (45942,)
Shape of y_test: (5105,)


## Part 4. Perform KNN Model Selection Using `GridSearchSV()`

Our goal is to find the optimal choice of hyperparameter $K$. We will then train a KNN model using that value of $K$.

### Set Up a Parameter Grid 

<b>Task</b>: Create a dictionary called `param_grid` that contains 10 possible hyperparameter values for $K$. The dictionary should contain the following key/value pair:

* A key called 'n_neighbors' 
* A value which is a list consisting of 10 values for the hyperparameter $K$ 

For example, your dictionary would look like this: `{'n_neighbors': [1, 2, 3,..]}`

The values for hyperparameter $K$  will be in a range that starts at $2$ and ends with $\sqrt{num\_examples}$, where `num_examples` is the number of examples in our training set `X_train`. Use the NumPy [np.linspace()](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) function to generate these values, then convert each value to an `int`.

In [4]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


num_examples = len(X_train)
start_k = 2
end_k = int(np.sqrt(num_examples))  
k_values = np.linspace(start_k, end_k, 10, dtype=int)


param_grid = {'n_neighbors': k_values}


knn = KNeighborsClassifier()


grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

# Fit GridSearchCV to find the best K value
grid_search.fit(X_train, y_train)


print("Best K value:", grid_search.best_params_['n_neighbors'])
print("Best cross-validation score:", grid_search.best_score_)


Best K value: 72
Best cross-validation score: 0.7105916161138512


### Perform Grid Search Cross-Validation

<b>Task:</b> Use `GridSearchCV` to search over the different values of hyperparameter $K$ to find the one that results in the best cross-validation (CV) score.

Complete the code in the cell below. <b>Note</b>: This will take a few minutes to run.

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

print('Running Grid Search...')


model = KNeighborsClassifier()


num_examples = len(X_train)
start_k = 2
end_k = int(np.sqrt(num_examples))  # Ending at sqrt(num_examples)
k_values = np.linspace(start_k, end_k, 10, dtype=int)
param_grid = {'n_neighbors': k_values}


grid = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')


grid_search = grid.fit(X_train, y_train)

print('Done')

Running Grid Search...
Done


<b>Task</b>: Retrieve the value of the hyperparameter $K$ for which the best score was attained. Save the result to the variable `best_k`. Print the result.

In [6]:

best_k = grid_search.best_params_['n_neighbors']

print("Best K value:", best_k)

Best K value: 72


## Part 5. Train the Optimal KNN Model and Make Predictions

<b>Task</b>: Initialize a `KNeighborsClassifier` model object with the best value of hyperparameter `K` and fit the model to the training data. The model object should be named `model_best`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_best = KNeighborsClassifier(n_neighbors=best_k)

model_best.fit(X_train, y_train)

<b>Task:</b> Test your model on the test set (`X_test`).

1. Use the ``predict_proba()`` method  to use the fitted model `model_best` to predict class probabilities for the test set. Note that the `predict_proba()` method returns two columns, one column per class label. The first column contains the probability that an unlabeled example belongs to class `False` (Churn is "False") and the second column contains the probability that an unlabeled example belongs to class `True` (Churn is "True"). Save the values of the *second* column to a list called ``probability_predictions``.

2. Use the ```predict()``` method to use the fitted model `model_best` to predict the class labels for the test set. Store the outcome in the variable ```class_label_predictions```. Note that the `predict()` method returns the class label (True or False) per unlabeled example.

In [15]:
from sklearn.model_selection import train_test_split


X = model_best  


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)



NameError: name 'y' is not defined

## Part 6. Evaluate the Accuracy of the Model

<b>Task</b>: Compute and print the model's accuracy score using `accuracy_score()`.

In [ ]:
from sklearn.metrics import accuracy_score


y_pred = model_best.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

<b>Task:</b> Create a confusion matrix to evaluate your model. Use the Confusion Matrix Demo as a reference.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


try:
    df = pd.read_csv('path_to_your_dataset.csv') 
except FileNotFoundError:
    print("File not found. Please check the file path and try again.")
    


if 'df' in locals():
    
    X = df.drop('Churn', axis=1)  
    y = df['Churn'] 

  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

   
    best_k = 5  
    model_best = KNeighborsClassifier(n_neighbors=best_k)
    model_best.fit(X_train, y_train)

    
    y_pred = model_best.predict(X_test)

   
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy Score: {accuracy}")

   
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(cm)

    
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

File not found. Please check the file path and try again.


## Part 7.  Plot the Precision-Recall Curve 

Recall that scikit-learn defaults to a 0.5 classification threshold. Sometimes we may want a different threshold. We can use the precision-recall curve to show the trade-off between precision and recall for different classification thresholds. Scikit-learn's `precision_recall_curve()` function computes precision-recall pairs for different probability thresholds. For more information, consult the [Scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html).

Let's first import the function.

In [ ]:
from sklearn.metrics import precision_recall_curve

<b>Task:</b> You will use `precision_recall_curve()` to compute precision-recall pairs. In the code cell below, call the function with the arguments `y_test` and `probability_predictions`. The function returns three outputs. Save the three items to the variables `precision`, `recall`, and `thresholds`, respectively. 


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load your dataset into a DataFrame 'df'
df = pd.read_csv('/path/to/your/dataset.csv')  # Adjust path as per your dataset location

# 2. Define features (X) and target variable (y)
X = df.drop('Churn', axis=1)  # Features: all columns except 'Churn'
y = df['Churn']  # Target variable: 'Churn' column

# Optionally, split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/dataset.csv'

The code cell below uses seaborn's `lineplot()` function to visualize the precision-recall curve. Variable `recall` will be on the $x$ axis and `precision` will be on the $y$-axis.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

sns.lineplot(x=recall, y=precision, marker = 'o')

plt.title("Precision-recall curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.show()